In [1]:
import pandas as pd
import numpy as np
from modules import constants

Read in full data filter, on D1 games only

In [27]:
def calculate_possessions(fga,ors,tos,fta) -> float:
    
    return fga - ors + tos + (0.475 * fta)

In [38]:
full_box_df = pd.read_parquet('parquet_files/box_scores_sports_reference_2024.gzip')
full_box_df['team_id'] = full_box_df.team_name.map(constants.TEAM_NAME_ID_DICT)
full_box_df['opp_id'] = full_box_df.opp_name.map(constants.TEAM_NAME_ID_DICT)
d1_df = full_box_df.loc[~full_box_df.opp_id.isin(constants.NON_D1_IDs)].copy()
d1_df = d1_df.loc[~d1_df.team_id.isin(constants.NON_D1_IDs)].reset_index(drop =True).copy()
# Coerce appropriate columns to numeric data types
for col in d1_df.columns.tolist():
    try:
        d1_df[col] = pd.to_numeric(d1_df[col])
    except ValueError as e:
        continue
assert d1_df.opp_id.nunique() == 362
assert d1_df.team_id.nunique() == 362
assert len(set(d1_df.team_id.unique().tolist() + d1_df.opp_id.unique().tolist())) == 362
d1_df.sample(5)

,date,opp_name,wl,score,opp_score,fg,fga,fgpct,3p,3pa,...,opp_orb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,team_name,team_id,opp_id
4251,2023-11-10,DAYTON,W,71,66,25,53,0.472,5,14,...,3,21,16,3,1,15,17,NORTHWESTERN,165,320
4384,2023-12-20,UNC,L,69,81,23,57,0.404,7,23,...,7,34,9,10,6,9,18,OKLAHOMA,98,108
220,2023-11-29,SAM HOUSTON,W,78,61,33,64,0.516,4,18,...,12,40,13,5,3,15,17,ARIZONA STATE,179,221
3864,2024-01-02,COLORADO STATE,L,68,76,28,60,0.467,8,17,...,7,36,14,10,6,15,14,NEW MEXICO,146,297
5745,2023-11-29,YALE,L,71,79,24,54,0.444,10,22,...,14,35,14,10,3,11,16,STONY BROOK,43,44


In [41]:
d1_df['possessions'] = d1_df.apply(lambda x: calculate_possessions(x.fga,x.orb,x.tov,x.fta),axis = 1)
d1_df['opp_possessions'] = d1_df.apply(lambda x: calculate_possessions(x.opp_fga,x.opp_orb,x.opp_tov,x.opp_fta),axis = 1)
d1_df['ortg'] = d1_df.apply(lambda x: 100*(x.score/x.possessions),axis =1)
d1_df['drtg'] = d1_df.apply(lambda x: 100*(x.opp_score/x.opp_possessions),axis =1)
d1_df['net_rtg'] = d1_df.apply(lambda x: x.ortg - x.drtg,axis = 1)
d1_df

,date,opp_name,wl,score,opp_score,fg,fga,fgpct,3p,3pa,...,opp_tov,opp_pf,team_name,team_id,opp_id,possessions,opp_possessions,ortg,drtg,net_rtg
0,2023-11-06,OKLAHOMA STATE,W,64,59,26,57,0.456,5,18,...,12,16,ABILENE CHRISTIAN,215,270,70.225,71.500,91.135635,82.517483,8.618153
1,2023-11-10,NC STATE,L,64,84,20,57,0.351,5,17,...,7,20,ABILENE CHRISTIAN,215,237,66.825,68.400,95.772540,122.807018,-27.034477
2,2023-11-14,PRAIRIE VIEW,L,74,79,23,60,0.383,5,19,...,14,23,ABILENE CHRISTIAN,215,333,74.775,71.150,98.963557,111.033029,-12.069471
3,2023-11-17,SAN JOSE STATE,W,77,71,25,59,0.424,7,15,...,6,18,ABILENE CHRISTIAN,215,139,69.350,63.700,111.031002,111.459969,-0.428966
4,2023-11-19,FORDHAM,W,59,45,22,55,0.400,3,12,...,19,16,ABILENE CHRISTIAN,215,65,66.075,66.975,89.292471,67.189250,22.103221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7043,2024-01-12,WRIGHT STATE,W,81,71,27,65,0.415,10,33,...,6,17,YOUNGSTOWN STATE,299,359,70.975,73.600,114.124692,96.467391,17.657300
7044,2024-01-17,OAKLAND,L,67,70,26,66,0.394,8,31,...,10,13,YOUNGSTOWN STATE,299,321,65.175,64.175,102.800153,109.076743,-6.276590
7045,2024-01-20,DETROIT MERCY,W,105,64,38,60,0.633,14,26,...,14,15,YOUNGSTOWN STATE,299,289,73.125,71.650,143.589744,89.323098,54.266645
7046,2024-01-25,IUPUI,W,78,50,31,61,0.508,4,20,...,13,16,YOUNGSTOWN STATE,299,186,73.650,73.500,105.906314,68.027211,37.879103


In [18]:
base_dict = {f"TM_{id}":0 for id in set(d1_df.team_id.unique().tolist() + d1_df.opp_id.unique().tolist())}
base_dict.update({f"OPP_{id}":0 for id in set(d1_df.team_id.unique().tolist() + d1_df.opp_id.unique().tolist())})

In [42]:
output_dict = {}
for meaningless_index_value,sub_dict in d1_df.to_dict(orient = 'index').items():
    
    output_dict[meaningless_index_value] = base_dict.copy()
    team_id = sub_dict['team_id']
    opp_id = sub_dict['opp_id']
    output_dict[meaningless_index_value][f"TM_{team_id}"] = 1
    output_dict[meaningless_index_value][f"OPP_{opp_id}"] = 1
    output_dict[meaningless_index_value]['ortg'] = sub_dict['ortg']
    output_dict[meaningless_index_value]['drtg'] = sub_dict['drtg']
    output_dict[meaningless_index_value]['pace'] = sub_dict['possessions']

In [47]:
reg_df = pd.DataFrame(output_dict).transpose()
for col in [i for i in reg_df.columns if (i.startswith("TM") or i.startswith("OPP"))]:
    reg_df[col] = reg_df[col].astype(int)
reg_df

,TM_0,TM_1,TM_2,TM_3,TM_4,TM_5,TM_6,TM_7,TM_8,TM_9,...,OPP_355,OPP_356,OPP_357,OPP_358,OPP_359,OPP_620,OPP_631,ortg,drtg,pace
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,91.135635,82.517483,70.225
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,95.772540,122.807018,66.825
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,98.963557,111.033029,74.775
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,111.031002,111.459969,69.350
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,89.292471,67.189250,66.075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7043,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,114.124692,96.467391,70.975
7044,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,102.800153,109.076743,65.175
7045,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,143.589744,89.323098,73.125
7046,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,105.906314,68.027211,73.650
